In [8]:
TRAIN_FILE='../wikitext-2-raw/wiki.train.raw'
TEST_FILE='../wikitext-2-raw/wiki.test.raw'

In [21]:
# Check that we have a GPU
!nvidia-smi

Fri Jul 10 20:48:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:17:00.0 Off |                  N/A |
| 23%   53C    P5    16W / 275W |     12MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:65:00.0 Off |                  N/A |
|  0%   39C    P8    10W / 275W |     29MiB / 11176MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [5]:
# Check that PyTorch sees it
import torch, gc
print(torch.cuda.is_available())

# print(torch.__version__)
# gc.collect()
# torch.cuda.empty_cache()

True
1.5.0


In [2]:
from transformers import RobertaConfig

config = RobertaConfig(
    hidden_size=128,
    num_attention_heads=1,
    num_hidden_layers=1,
    type_vocab_size=1, 
    intermediate_size=3072,
    max_position_embeddings=128
)

In [14]:
from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config=config) #this returns error cuda runtime error (700) : an illegal memory access was encountered at /pytorch/aten/src/THC/THCCachingHostAllocator.cpp:278
#model = RobertaForMaskedLM.from_pretrained('roberta-base') #this works

In [15]:
model.num_parameters()

4843578

In [16]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [17]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=TRAIN_FILE,
    block_size=12,
)

CPU times: user 10.6 s, sys: 60.3 ms, total: 10.7 s
Wall time: 10.7 s


In [18]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [19]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./RoBERTa",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    save_steps=10000,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [20]:
%%time
trainer.train()

RuntimeError: cuda runtime error (700) : an illegal memory access was encountered at /pytorch/aten/src/THC/THCCachingHostAllocator.cpp:278